In [1]:
import requests as r
import numpy as np
import pandas as pd
import bs4
from selenium import webdriver
import os
import time
from tqdm import tqdm

### Scrape business ids from yelp

In [2]:
bookmarks = 'https://www.yelp.com/collection/Mr2soK2jCZSbUNWr25lvCQ'

options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-extensions")
options.add_argument("--headless")

browser = webdriver.Chrome(executable_path=os.path.abspath('../chromedriver/chromedriver.exe'), options=options)
browser.get('https://www.yelp.com/collection/Mr2soK2jCZSbUNWr25lvCQ')
# browser.execute_script("window.scrollTo(0, Y)") 

browser.execute_script("document.getElementsByClassName('collection-content collection-left-" + 
                       "pane u-inline-block')[0].scrollTop += 100000");

time.sleep(5)

soup = bs4.BeautifulSoup(browser.page_source, 'html.parser')

business_ids = [broth.get('href').split('/')[-1] for broth in 
                        soup.find_all("a", {"class": "biz-name js-analytics-click"})]

browser.quit()

### Use yelp API to get info on each bookmarked business

In [3]:
url = "https://api.yelp.com/v3/businesses/matches"
key = "enieurwIwfub1aieeynJjuvOXhqAIY1wUJsbHCNjZBgBoAWDozNv60h3g-3MhUcuRl1tzFWedI7ME7w3aCYmEHoNyQOyQFZOPgO9aXo1H4H3G7vjgc7t0zzIPfr8YHYx"
headers = {'Authorization': 'bearer %s' % key}

In [4]:
business_info = []
for business_id in tqdm(business_ids):
    business_info.append(r.get(f'https://api.yelp.com/v3/businesses/{business_id}', headers=headers).json())
df = pd.DataFrame(business_info)

100%|████████████████████████████████████████████████████████████████████████████████| 116/116 [00:48<00:00,  2.41it/s]


In [17]:
reviews = {}
for ind, row in tqdm(df.iterrows()):
    response = r.get(row.url)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    reviews[ind] = [review.text for review in soup.find_all('p', {'class':'comment__373c0__Nsutg css-n6i4z7'})]
df['reviews'] = df.index.map(reviews)

116it [06:21,  3.29s/it]


In [22]:
df.to_csv('data/bookmarked_restaurants.csv', index=False)